In [2]:
import sys
import math
import random
from collections import defaultdict
import matplotlib.pyplot as plt

# asdfasdf

In [3]:
def sampleOne(probs):
    z = 0
    for k, v in probs.items():
        z = z + v
    remaining = random.uniform(0, z)
    for k,v in probs.items():
        remaining = remaining - v
        if  remaining <= 0:
            return k


In [21]:
p ={"A": 0.5, "B":0.25, "C":0.25}
mydic = defaultdict(lambda :0)
sample_num = 1000000
for i in range(sample_num):
    mydic[sampleOne(p)] +=1
for k, v in mydic.items():
    print(k, v)
# print (len(mydic))
for k, v in sorted(mydic.items()):
    print("p[%s]=%0.2f"%(k, v/sample_num))

C 249738
A 500784
B 249478
p[A]=0.50
p[B]=0.25
p[C]=0.25


In [10]:
class MyClass:
    """A simple example class"""         # 三重クォートによるコメント

    def __init__(self):                  # コンストラクタ
        self.name = ""

    def getName(self):                   # getName()メソッド
        return self.name

    def setName(self, name):             # setName()メソッド
        self.name = name

In [13]:
obj = MyClass()

In [ ]:
obj.name